In [1]:
import os
import sys
sys.path.append('E:/DA')
from pysql import SQLite
sql = SQLite(database='E:/DA/database/t.db')

In [2]:
# 查询 所有 课程成绩 小于 60 的 学生的学号和姓名
# 思路 使用 exists 关键字 排除 存在及格情况的学生
query = '''
select t1.s_id, t1.s_name from (
    score s left join student as stu on s.s_id = stu.s_id
) as t1
where not exists (
    select *
    from score as t2
    where t1.s_id = t2.s_id and t2.s_score >= 60
) 
group by t1.s_id ;
'''
sql.read_sql(query)

,s_id,s_name
0,04,李云
1,06,吴兰


In [5]:
# 问题: 查询所有学生的学号、姓名、选课数、总成绩
query = '''
select stu.s_id as 学号, 
       stu.s_name as 姓名, 
       count(sco.c_id) as '选课数', 
       sum(case when sco.s_score is null then 0 else sco.s_score end) as '总成绩'
from student as stu
left join score as sco
on stu.s_id = sco.s_id
group by stu.s_id;
'''
sql.read_sql(query)

,学号,姓名,选课数,总成绩
0,01,赵雷,3,269
1,02,钱电,3,210
2,03,孙风,3,240
3,04,李云,3,100
4,05,周梅,2,163
5,06,吴兰,2,65
6,07,郑竹,2,187
7,08,王菊,0,0


In [6]:
# 问题: 查询姓“张”的老师的个数
query='''
select t_name as '姓名', count(t_id) as 个数
from teacher
where t_name like '张%';
'''
sql.read_sql(query)

,姓名,个数
0,张三,1


In [8]:
# 问题: 查询没学过“张三”老师课的学生的学号、姓名
query='''
select s_id, s_name from student
where s_id not in (
    select s.s_id from score as s
    inner join course as c on c.c_id=s.c_id
    inner join teacher as t on t.t_id=c.t_id
    where t.t_name='张三'
);
'''
sql.read_sql(query)

,s_id,s_name
0,06,吴兰
1,08,王菊


In [34]:
sql.read_sql('''
SELECT st.s_id,st.s_name FROM student AS st 
INNER JOIN score AS s ON st.s_id=s.s_id
INNER JOIN course AS c ON s.c_id=c.c_id
INNER JOIN teacher AS t ON t.t_id=c.t_id
WHERE t.t_name ='张三' 
GROUP BY st.s_id
HAVING
COUNT(st.s_id) IN
(SELECT COUNT(c_name) FROM course 
INNER JOIN teacher 
ON course.t_id=teacher.t_id 
WHERE teacher.t_name='张三')
''')

,s_id,s_name
0,01,赵雷
1,02,钱电
2,03,孙风
3,04,李云
4,05,周梅
5,07,郑竹


In [3]:
sql.read_sql('''
select s_id,s_name from student
where s_id in (
    select s1.s_id from (
        select * from score where c_id = '01') as s1
    inner join (
        select * from score where c_id = '02') as s2
    on s1.s_id = s2.s_id
);
''')

,s_id,s_name
0,01,赵雷
1,02,钱电
2,03,孙风
3,04,李云
4,05,周梅


In [15]:
query = '''
SELECT c_id '课程编号',
       sum(s_score) '总成绩',
       avg(s_score) '平均成绩',
       count(s_score) as '选课人数'
from score
group by c_id;
'''
sql.read_sql(query)

,课程编号,总成绩,平均成绩,选课人数
0,01,387,64.500000,6
1,02,436,72.666667,6
2,03,411,68.500000,6


In [26]:
query = '''
select st.s_id '学生学号',st.s_name '学生姓名'
from student as st
left join score as sc on st.s_id=sc.s_id
group by st.s_id 
having count(distinct sc.c_id) < (select count(distinct c_id) from course);
'''
sql.read_sql(query)

,学生学号,学生姓名
0,05,周梅
1,06,吴兰
2,07,郑竹
3,08,王菊


In [27]:
query='''
select s_id,s_name from student 
where s_id in (select distinct s_id FROM score 
               where c_id in (select c_id from score where s_id='01')
               and s_id!='01')'''
sql.read_sql(query)            

,s_id,s_name
0,02,钱电
1,03,孙风
2,04,李云
3,05,周梅
4,06,吴兰
5,07,郑竹


In [43]:

query='''
SELECT s_id FROM score
WHERE s_id <>'01'
GROUP BY s_id
HAVING GROUP_CONCAT(distinct c_id)=(
SELECT GROUP_CONCAT(distinct c_id) FROM score WHERE s_id='01')
'''
sql.read_sql(query)            

,s_id
0,02
1,03
2,04


In [ ]:

# -- 学生表
# -- Student（s_id，s_name，s_birth，s_sex）
# -- 学生编号,学生姓名, 出生年月,学生性别

# -- 课程表
# -- Course(c_id,c_name,t_id) 
# -- 课程编号, 课程名称, 教师编号

# -- 教师表
# -- Teacher(t_id,t_name)
# -- 教师编号,教师姓名

# -- 成绩表
# -- Score(s_id,c_id,s_s_score) 
# -- 学生编号,课程编号,分数
